## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import setuptools
import pathlib
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2024-09-25 23:37:20.506885: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 23:37:20.507219: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 23:37:20.509163: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 23:37:20.514546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 23:37:20.523062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME', 'STATUS'], inplace = True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
df_vc_app_type = application_df['APPLICATION_TYPE'].value_counts()
df_vc_app_type

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = df_vc_app_type[df_vc_app_type < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
df_vc_clss_type = application_df['CLASSIFICATION'].value_counts()
df_vc_clss_type

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
df_vc_clss_type_gt1 = df_vc_clss_type[df_vc_clss_type > 1]
df_vc_clss_type_gt1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = df_vc_clss_type[df_vc_clss_type < 500].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [9]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT',
       'IS_SUCCESSFUL'],
      dtype='object')

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_from = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
df_dummies = pd.get_dummies(application_df[dummies_from])
df_dummies.head(3)

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False


In [11]:
non_dummies = ['ASK_AMT']
df_non_dummies = application_df[non_dummies]
df_non_dummies.head(3)

,ASK_AMT
0,5000
1,108590
2,5000


In [12]:
# Split our preprocessed data into our features and target arrays
X = pd.concat([df_non_dummies, df_dummies], axis=1)

y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=12, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 253us/step - accuracy: 0.6766 - loss: 0.6192
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step - accuracy: 0.7279 - loss: 0.5576
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 230us/step - accuracy: 0.7306 - loss: 0.5540
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step - accuracy: 0.7313 - loss: 0.5517
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step - accuracy: 0.7340 - loss: 0.5514
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.7356 - loss: 0.5475
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 259us/step - accuracy: 0.7319 - loss: 0.5493
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 227us/step - accuracy: 0.7329 - loss: 0.5482
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step - accuracy: 0.7362 - loss: 0.5438
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step - accuracy: 0.7360 - loss: 0.5443
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step - accuracy: 0.7314 - loss: 0.5458
Epoch 12/150
804/80

# Manual optimization

## Additional dropped columns

Status dropped due to extemely low support values.

Dropping special_considerations reduced accuracy signifcantly (0.01). Dropping other columns were worse.

Classifcation binning from < 500 to < 1000 or < 250 reduced perf by 0.01

Application type binning from 500 to 1000 reduced perf by 0.008

## Activations

* relu - best in single later 0.735
* tanh - good but relu outperforms 0.730

## Layers and units

* single layer best accuracy was 0.735, 8 units relu
* 8 neurons perform better than 4 or 16 in single layer
* 4, 16 unit additional layer performed worse when paired with 8 layers before or after.
* single 64 unit layer acc: 0.731

## Epochs
Tested 25, 50, 100, 150, and 250 with a 8-8 2 layer relu model. 150 was best.

## Best accuracy
* 16-12 relu functioned best so far. 0.738

In [17]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 366us/step - accuracy: 0.7272 - loss: 0.5531
Loss: 0.5531340837478638, Accuracy: 0.7272303104400635


# Automated optimazation

In [18]:
def create_model(hp):
    model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activations = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    hp_units1 = hp.Int('units_1', min_value = 64, max_value = 256, step = 32)
    model.add(tf.keras.layers.Dense(units=hp_units1, activation=hp.Choice('activation',['relu','tanh', 'sigmoid'])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    hp_units2 = hp.Int('units_2', min_value = 16, max_value = 64, step = 8)
    model.add(tf.keras.layers.Dense(units=hp_units2, activation=hp.Choice('activation',['relu','tanh', 'sigmoid'])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    hp_units2 = hp.Int('units_2', min_value = 4, max_value = 10, step = 2)
    model.add(tf.keras.layers.Dense(units=hp_units2, activation=hp.Choice('activation',['relu','tanh', 'sigmoid'])))

    # Add the final layer, for binary classifcation
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))#hp.Choice('activation',['sigmoid','softmax'])))

    # Compile the model
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return model

In [19]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=200,
    hyperband_iterations=2
)

Reloading Tuner from ./untitled_project/tuner0.json


In [20]:
tuner.search(X_train_scaled, y_train, epochs=150, validation_data=(X_test_scaled, y_test))

In [21]:
top_models = tuner.get_best_models(3)
for model in top_models:
    model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/deep-learning-challenge/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


268/268 - 0s - 527us/step - accuracy: 0.7259 - loss: 0.5481
Loss: 0.5481114983558655, Accuracy: 0.7259474992752075
268/268 - 0s - 504us/step - accuracy: 0.7249 - loss: 0.5500
Loss: 0.5499818325042725, Accuracy: 0.7248979806900024
268/268 - 0s - 500us/step - accuracy: 0.7283 - loss: 0.5463
Loss: 0.54633629322052, Accuracy: 0.7282798886299133


In [22]:
top_model = tuner.get_best_models(1)[0]

In [23]:
model_loss, model_accuracy = top_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - 528us/step - accuracy: 0.7259 - loss: 0.5481
Loss: 0.5481114983558655, Accuracy: 0.7259474992752075


In [24]:
top_hyper = tuner.get_best_hyperparameters(1)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'units_1': 224, 'units_2': 56, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 4, 'tuner/round': 0}


# Run One more attempt with checkpoints and callbacks

and much larger neuron counts

In [25]:
nn_tuned = tf.keras.models.Sequential()

# First hidden layer
nn_tuned.add(tf.keras.layers.Dense(units=128, activation="relu"))

# Second hidden layer
nn_tuned.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Third hidden layer
nn_tuned.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn_tuned.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_tuned.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Compile the model
nn_tuned.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
checkpoint_path = "training/cp.weights.h5"
checkpoint_dir = pathlib.Path.cwd() / checkpoint_path

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq='epoch',
                                                 verbose=1)

# Epochs reduced

75 epochs gives a 0.03% reduction in performance compared to 200.

In [35]:
# Train the model
fit_model_tuned = nn_tuned.fit(X_train_scaled, 
                               y_train, 
                               epochs=75, 
                               callbacks=[cp_callback])

Epoch 1/75


793/804 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step - accuracy: 0.7397 - loss: 0.5320
Epoch 1: saving model to training/cp.weights.h5
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step - accuracy: 0.7398 - loss: 0.5319
Epoch 2/75
802/804 ━━━━━━━━━━━━━━━━━━━━ 0s 313us/step - accuracy: 0.7429 - loss: 0.5268
Epoch 2: saving model to training/cp.weights.h5
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - accuracy: 0.7429 - loss: 0.5268
Epoch 3/75
651/804 ━━━━━━━━━━━━━━━━━━━━ 0s 309us/step - accuracy: 0.7443 - loss: 0.5244
Epoch 3: saving model to training/cp.weights.h5
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step - accuracy: 0.7440 - loss: 0.5252
Epoch 4/75
675/804 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.7453 - loss: 0.5282
Epoch 4: saving model to training/cp.weights.h5
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step - accuracy: 0.7452 - loss: 0.5279
Epoch 5/75
662/804 ━━━━━━━━━━━━━━━━━━━━ 0s 303us/step - accuracy: 0.7464 - loss: 0.5240
Epoch 5: saving model to training/cp.weights.h5
804/804 ━━━━━━━━━━━━━━━

In [36]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn_tuned.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 212us/step - accuracy: 0.7262 - loss: 0.6403
Loss: 0.6403024792671204, Accuracy: 0.7261807322502136


In [30]:
# Export our model to HDF5 file
export_name = "AlphabetSoupCharity.h5"
nn_tuned.save(export_name, overwrite=True)